# **Especialização em Ciência de Dados - INF/UFRGS e SERPRO**
### Disciplina CD004 - Metodologia de Aprendizado de Máquina Supervisionado
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br> 

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de diferentes algoritmos e estratégias de Aprendizado de Máquina.*


---


<br>

## **Aula 03** - **Tópico: Pré-processamento de dados. Imputação e Transformação.**

<br>

**Objetivo deste notebook**: Explorar estratégias de pré-processamento de dados, incluindo imputação de valores e transformação de dados (codificação, discretização, normalização). Conhecer e compreender a utilidade de Pipelines em scikit-learn/Python.
<br>

---




##**Exemplos simples de imputação, codificação, discretização de dados com Python**
Primeiramente, vamos exemplificar o funcionamento básico dos métodos do scikit-learn voltados para a imputação de dados e transformação de dados. É perceptível a semelhança com o uso dos métodos para normalização de dados que temos explorado nas nossas aulas. A seguir revisamos um exemplo simples de normalização de dados numéricos com StandardScaler() e MinMaxScaler(). 

### Revisando a normalização de dados

In [ ]:
## Carregando as bibliotecas básicas necessárias
# A primeira linha é incluída para gerar os gráficos logo abaixo dos comandos de plot
%matplotlib inline              
import pandas as pd             # para análise de dados 
import matplotlib.pyplot as plt # para visualização de informações
import seaborn as sns           # para visualização de informações
import numpy as np              # para operações com arrays multidimensionais

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

## gera um conjunto de dados sintético para classificação
## o conjunto de dados contém 40 instâncias e 5 atributos
## o problema de classificação é binário
X, y = make_classification(n_samples=40,n_features=5,random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,test_size=0.2)

## O scikit-learn disponibiliza diferentes métodos para normalização/padronização
#scaler = StandardScaler()
scaler = MinMaxScaler()


## para os dados de treino podemos primeiro chamar .fit() e depois .transform()
## ou podemos substituir ambas por uma única chamada .fit_transform().
## CUIDADO: fit_transform() NÂO deve ser usados para dados de validação/teste
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)

In [ ]:
print(X_test)

In [ ]:
print(X_test_scaled)

### Imputação de valores faltantes

Embora as células a seguir lidem com a imputação para valores faltantes, podemos utilizar as mesmas estratégias de imputação para corrigir ruídos/outliers.

In [ ]:
## fazemos uma cópia dos dados para poder simular valores faltantes
X_train_missing = X_train.copy()
X_test_missing = X_test.copy()

In [ ]:
import random

## simula alguns valores faltantes, alterando posições aleatórias dos conjuntos
## de treinamento e teste

X1=random.sample(range(X_train.shape[0]), 8)
X2=random.sample(range(X_train.shape[0]), 5)
print(X1)
print(X2)

X_train_missing[X1,0] = np.nan
X_train_missing[X2,3] = np.nan


X3=random.sample(range(X_test.shape[0]), 2)
X4=random.sample(range(X_test.shape[0]), 1)
print(X3)
print(X4)

X_test_missing[X3,0] = np.nan
X_test_missing[X4,3] = np.nan

In [ ]:
## verificando o conjunto de teste após simulação de valores faltantes
X_test_missing

In [ ]:
## realização a imputação de valores com SimpleImputer
## nos atributos (todos numéricos)
from sklearn.impute import SimpleImputer

## Inicializa um SimpleImputer que irá substituir valores faltantes pela média dos dados
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

## Estima o parâmetro (média) a partir dos dados de treino e transforma os dados de treino
X_train_missing_fix1= imp_mean.fit_transform(X_train_missing)

## Transforma os dados de teste
X_test_missing_fix1 = imp_mean.transform(X_test_missing)
print(X_test_missing_fix1)

In [ ]:
from sklearn.impute import KNNImputer


## Inicializa um KNNImputer que irá substituir valores faltantes pelo valor estimado com um KNN
imp_knn = KNNImputer(n_neighbors=2, weights="uniform")

## Estima o parâmetro a partir dos dados de treino e transforma os dados de treino
X_train_missing_fix2 = imp_knn.fit_transform(X_train_missing)

## Transforma os dados de teste
X_test_missing_fix2  = imp_knn.transform(X_test_missing)
print(X_test_missing_fix2)

In [ ]:
## Para valores categóricos, podemos usar o SimpleImputer com 
## a moda (most_frequent) ou valor constante
df1 = pd.DataFrame([["a", "x"],
                  [np.nan, "y"],
                  ["a", np.nan],
                  ["b", "y"]], dtype="category")
print(df1)

## Imputação com a moda
imp_mode = SimpleImputer(strategy="most_frequent")
print(imp_mode.fit_transform(df1))


## Imputação com um valor padrão
imp_mode = SimpleImputer(strategy="constant",fill_value='?')
print(imp_mode.fit_transform(df1))

### Discretização de dados numéricos

As células abaixo demonstram diferentes estratégias para discretizar dados numéricos ('quantile' e 'uniform'), bem como diferentes formas de codificar o resultado da discretização ('ordinal' e 'onehot-dense').

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
discr_uni = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
#discr_uni = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
#discr_uni = KBinsDiscretizer(n_bins=3, encode='onehot-dense', strategy='uniform')

## Faz uma cópia dos dados originais de treino e teste
X_train_cp1 = X_train.copy()
X_test_cp1 = X_test.copy()

## Estima os parâmetros da discretização de dados a partir dos dados de treino e transforma os dados de treino
X_train_cp1 = discr_uni.fit_transform(X_train)

## Transforma os dados de teste
X_test_cp1 = discr_uni.transform(X_test)

In [ ]:
print(X_test_cp1)

### Codificação de dados categóricos

> Bloco com recuo



A codificação (encoding) de valores categóricos consiste em transformá-los para uma representação numérica. Essa transformação é muito útil para lidar com dados que possuam tipos mistos de atributos, bem como para utilizar algoritmos que possuam a restrição de aceitar como entrada apenas valores numéricos.

In [ ]:
## Simulando exemplos categóricos para análise
X1 = np.array([['dog'] * 5 + ['cat'] * 10 + ['rabbit'] * 7 + ['snake'] * 3], dtype=object).T
X2 = np.array([['male'] * 2 + ['female'] * 10 + ['male'] * 8 + ['female'] * 5], dtype=object).T

X_train_cat = np.concatenate((X1,X2), axis=1)
X_test_cat = np.array([[ 'dog', 'male'],
                      [ 'cat',  'female'],
                      [ 'rabbit', 'male']])

In [ ]:
print(X_train_cat)

In [ ]:
print(X_test_cat)

Exemplo do uso de OrdinalEncoder(). Cada categoria é mapeada para um valor inteiro, de 0 até [número categorias - 1].

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
enc_ord = OrdinalEncoder()

X_train_cat_ord = enc_ord.fit_transform(X_train_cat)
X_test_cat_ord  = enc_ord.transform(X_test_cat)

In [ ]:
print(X_test_cat_ord)

Tendo em vista que o OrdinalEncoder() substitui cada valor categórico por um inteiro, não é recomendado para atributos que sejam nominais, isto é, que não possuem relação de ordem implícita. Para atributos categóricos nominais, é mais recomendado utilizar OneHotEncoder(). A opção Sparse=False determina que será gerado um array com 0's e 1's: cada categoria possível para um determinado atributo vira um novo atributo binário.


In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc_ohe = OneHotEncoder(sparse=False)

X_train_cat_ohe = enc_ohe.fit_transform(X_train_cat)
X_test_cat_ohe = enc_ohe.transform(X_test_cat)

In [ ]:
## visualiza o resultado da codificação 
print(X_test_cat_ohe)

In [ ]:
## Ajusta o nome dos atributos como species_'nomecatX'
print(enc_ohe.get_feature_names_out(['species','gender']))

Para atributos categóricos binários, é possível remover uma das colunas após a codificação.

In [ ]:
enc_ohe = OneHotEncoder(sparse=False,drop='if_binary')

X_train_cat_ohe = enc_ohe.fit_transform(X_train_cat)
X_test_cat_ohe = enc_ohe.transform(X_test_cat)

In [ ]:
print(X_test_cat_ohe)

In [ ]:
print(enc_ohe.get_feature_names_out(['species','gender']))

Em alguns domínios, o atributo categórico pode ter muitas categorias possíveis, aumentando muito a dimensionalidade do problema após a codificação. Nestes casos, pode ser útil mapear para novos atributos apenas as categorias mais frequentes, seja determinando um valor fixo de número de categorias (max_categories) ou determinando uma frequência mínima nos dados (min_frequency). As categorias infrequentes serão agregadas em uma única coluna após o processo de codificação. **(Esta opção não está disponível na versão '0.24.1' do scikit-learn,instalada por padrão no Google Colab)**



##**Predição de 'churn' de clientes de serviço de telecomunicação**

Tendo demonstrado o funcionamento básico de estratégias de preparação de dados, vamos aplicá-las a um conjunto de dados mais interessante.

Os dados que utilizaremos neste notebook se referem a uma empresa de telecomunicações fictícia que forneceu serviços de telefone residencial e Internet para clientes na Califórnia. O conjunto de dados possui informações sobre os serviços para os quais cada cliente se inscreveu (telefone, várias linhas, internet, segurança online, backup online, proteção de dispositivos, suporte técnico e streaming de TV e filmes), informações da conta do cliente (há quanto tempo eles são clientes, contrato, forma de pagamento, cobrança sem papel, cobranças mensais e cobranças totais) e informações demográficas sobre os clientes (sexo, faixa etária, se eles têm parceiros e dependentes). Há, ainda, uma coluna chamada 'churn', que indica os clientes que desistiram do contrato do serviço no último mês. O objetivo da tarefa preditiva é identificar o churn (i.e., saída) de clientes a partir das informações coletadas. Os dados podem ser acessados neste [link](https://www.kaggle.com/datasets/blastchar/telco-customer-churn).




---



###Carregando e inspecionando os dados

Primeiramente, vamos carregar algumas bibliotecas importantes do Python e os dados a serem utilizados neste estudo. Os dados são disponibilizados através de um link, que também pode ser diretamente acessado pelos alunos.

In [ ]:
## Bibliotecas para treinamento/avaliação de modelos
from sklearn.model_selection import RepeatedKFold, train_test_split, cross_validate, cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

sns.set()

In [ ]:
!pip install missingno ## instalando biblioteca para visualizar valores faltantes

In [ ]:
import missingno as msno

In [ ]:
## Carregando os dados
df = pd.read_csv("https://drive.google.com/uc?export=view&id=10VrzI8mA2wPvkNIDaLzv-IglPU-Febtf",na_values=["NA"])
df  

In [ ]:
## Características gerais do dataset
print("O conjunto de dados possui {} linhas e {} colunas".format(df.shape[0], df.shape[1]))

In [ ]:
## Existem dados duplicados?
df.drop_duplicates(keep='last').shape

A coluna *'Churn'* contém a classificação de cada instância. Vamos avaliar a distribuição de classes do problema.

In [ ]:
## Distribuição do atributo alvo
plt.hist(df['Churn'])
plt.title("Distribuição do atributo alvo")
plt.show()

Quais os tipos de atributos nos dados? Os dados possuem valores faltantes?
Podemos responder estas perguntas utilizando o método .info() para um dataframe.

In [ ]:
df.info()

A biblioteca missingno permite visualizar os dados faltantes como uma matriz. Outras opções de visualização estão disponíveis e podem ser consultadas na documentação da biblioteca.

In [ ]:
msno.matrix(df)

O atributo customerID é único para cada instância e no geral não possui valor preditivo. Iremos removê-lo da análise.

In [ ]:
customer_ids = df['customerID']
df = df.drop(['customerID'], axis=1)

In [ ]:
##remove as duplicatas
df=df.drop_duplicates(keep='last')

Como atributos categóricos e numéricos podem demandar pré-processamentos diferentes, vamos separar os respectivos "nomes" em dois vetores distintos, facilitando a manipulação dos dados posteriormente.

In [ ]:
## Separa os atributos em vetores, de acordo com o tipo de dado (categórico ou numérico)
cat_columns=list(df.drop(['Churn'], axis=1).select_dtypes(include=["object"]).columns)
print(cat_columns)

num_columns=list(df.select_dtypes(include=["int64", "float64"]).columns)
print(num_columns)

É sempre importante inspecionar os dados. Em relação a atributos categóricos, podemos inspecionar os valores que cada um pode assumir, quantas instâncias temos por valor de atributo, distribuição destes entre classes (não inspecionado nas células a seguir), etc. Em relação a atributos numéricos, é importante observar a distribuição de valores e se há ocorrência de outliers. O boxplot, por padrão, mostra outliers de acordo com o método do IQR (valores que estão 1.5*IQR acima do terceiro quartil ou abaixo do primeiro quartil).

In [ ]:
def dist_plot(df,columns,type='boxplot'):
    plt.figure(figsize=(16, 5))
    for indx, var  in enumerate(columns):
        plt.subplot(1, 3, indx+1)
        if (type=='boxplot'):
          g = sns.boxplot(x=var, data=df,showfliers=True)
        else: 
          if (type=='histogram'):
            g = sns.histplot(x=var, data=df)
    plt.tight_layout()

def count_plot(df,columns):
    plt.figure(figsize=(20, 12))
    for indx, var  in enumerate(columns):
        plt.subplot(6, 3, indx+1)
        g = sns.countplot(x=var, data=df)
    plt.tight_layout()

In [ ]:
count_plot(df,cat_columns)

Alguma variável categórica pode ser interpretada como categórica ordinal, ou todas são categóricas nominais? Esta distinção é importante para avaliar o tipo de codificação a ser aplicada na transformação dos dados.

In [ ]:
dist_plot(df,num_columns)#,type="histogram")

O boxplot indica a existência de outliers (univariados) no conjunto de dados?


---


### Criando conjuntos de treino e teste para avaliação de modelos


Antes de iniciar o treinamento do modelo, lembre-se que é recomendado sempre reservar uma porção dos dados para teste, a qual somente será utilizada para avaliação do modelo final (após todo o processo de treinamento e otimização de hiperparâmetros).

Vamos fazer esta divisão, separando 20% para teste. Entretanto, primeiro precisamos dividir os dados entre atributos (X) e classe (y). Também iremos codificar as classes Yes/No para 1/0.



In [ ]:
## Separa o dataset em duas variáveis: os atributos/entradas (X) e a classe/saída (y)
X = df.drop(['Churn'], axis=1)
y = df['Churn'].values

Faremos o mapeamento das classes Yes/No para 1/0. Por padrão, as funções de avaliação assumem que a classe 1 é a positiva/de interesse.

In [ ]:
## substitui No' por 0, 'Yes' por 1
y = np.array([0 if y=='No' else 1 for y in y]) 

In [ ]:
## Faz a divisão entre treino (80%) e teste (20%).
## O conjunto de treino representa os dados que serão usados
## ao longo do desenvolvimento do modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,stratify=y,random_state=42) 

---


## Lidando com valores faltantes

Na análise exploratória pudemos perceber que temos alguns valores faltantes para duas variáveis, 'tenure' e 'gender'. Precisamos definir como tratar estes valores faltantes: removemos a instância, imputamos com um valor constante, imputamos com uma estatística, imputamos usando uma estimador multivariado (por exemplo, um KNN)? Abaixo vamos ver algumas estratégias possíveis. 

As células abaixo sumarizam os valores faltantes após divisão de treino/teste.

In [ ]:
msno.matrix(pd.DataFrame(X_train))

In [ ]:
pd.DataFrame(X_train).info()

In [ ]:
pd.DataFrame(X_test).info()

Uma opção seria simplesmente remover as instâncias com valores faltantes. Em domínios com muitos dados e pouca proporção de valores faltantes, esta remoção normalmente não terá implicações negativas. A célula abaixo mostra como isso poderia ser feito, salvando os novos dados em 'df2'

In [ ]:
## para remover as instâncias com valores faltantes 
## (neste caso pode ser aplicado a todo o conjunto de dados)
df2 = df.dropna(inplace=False)
print("O conjunto de dados após remoção de valores faltantes possui {} linhas e {} colunas".format(df2.shape[0], df2.shape[1]))

Não existem muitos valores faltantes. Ainda assim, os algoritmos em Python demandam que os valores faltantes sejam tratados de alguma forma. Uma alternativa à remoção das instâncias com valores faltantes é explorar estratégias de imputação de valores para substituí-los por algum outro valor simbólico dentro da distribuição original dos dados. Como temos atributos numéricos e categóricos, o valor a ser substituído (ou a estratégia para definir o valor) depende do tipo de dado. 

Os 'imputers' definidos abaixo lidam com esta especificidade. Vamos explorar o uso do SimpleImputer, capaz de lidar com valores numéricos ou categóricos.

Perceba que o método ColumnTransformer permite aplicar diferentes tipos de pré-processamento a diferentes grupos de atributos (colunas).

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

## para o atributo numérico podemos usar a mediana
imputer_Median = SimpleImputer(strategy='median')

## para o atributo categórico, podemos usar a moda (valor mais frequente)
imputer_Mode = SimpleImputer(strategy='most_frequent')

## Cada "imputer" deve ser ajustado (.fit())ao seu respectivo tipo de dado 
## Uma forma prática de fazer isto no Python é usar o ColumnTransformer
## O ColumnTransformer permite aplicar diferentes transformações a colunas específicas

col_imputer = ColumnTransformer(transformers=[('imputer_median', imputer_Median, num_columns),
                                               ('imputer_mode',  imputer_Mode, cat_columns)],
                                               remainder='passthrough')
## 'ajustamos' os parâmetros de imputação a partir dos dados de treinamento
col_imputer.fit(X_train)

## aplicamos para imputação no treino e no teste
X_train_imp1 = col_imputer.transform(X_train)
X_test_imp1 = col_imputer.transform(X_test)

In [ ]:
## De acordo com a documentação:
# The order of the columns in the transformed feature matrix follows the order of 
# how the columns are specified in the transformers list. 

print(col_imputer.transformers_[0][2])
print(col_imputer.transformers_[1][2])

In [ ]:
# ajusta nome das columas e mostra dataframe
columns = np.append(num_columns,cat_columns)
display(pd.DataFrame(X_train_imp1, columns=columns))

Verificando o resultado através da inspeção dos dados e contagem de valores não-nulos

In [ ]:
msno.matrix(pd.DataFrame(X_train_imp1))

In [ ]:
pd.DataFrame(X_train_imp1).info()

In [ ]:
pd.DataFrame(X_test_imp1).info()

Outra possibilidade é usar o KNNImputer, entretanto, o mesmo possui a limitação de esperar como entrada apenas valores numéricos. Para valores categóricos, os mesmos precisariam ser convertidos em numéricos através de alguma estratégia de codificação. Como os dados ainda não foram transformados, usaremos o ColumnTransformer para exemplificar a aplicação do KNNImputer apenas nos atributos numéricos.

In [ ]:
from sklearn.impute import KNNImputer

## para o atributo numérico podemos usar o KNNImputer
imputer_KNN = KNNImputer(n_neighbors=3, weights="uniform")

## para o atributo categórico, podemos usar a moda com SimpleImputer
imputer_Mode = SimpleImputer(strategy='most_frequent')

## Cada "imputer" deve ser ajustado (.fit()) ao seu respectivo tipo de dado 
## Uma forma prática de fazer isto no Python é usar o ColumnTransformer
## O ColumnTransformer permite aplicar diferentes transformações a colunas específicas

col_imputer = ColumnTransformer(transformers=[('imputer_KNN', imputer_KNN, num_columns),
                                               ('imputer_mode',  imputer_Mode, cat_columns)])
## 'ajustamos' os parâmetros de imputação a partir dos dados de treinamento
col_imputer.fit(X_train)

## aplicamos para imputação no treino e no teste
X_train_imp2 = col_imputer.transform(X_train)
X_test_imp2 = col_imputer.transform(X_test)

In [ ]:
# ajusta nome das columas e mostra dataframe
columns = np.append(num_columns,cat_columns)
display(pd.DataFrame(X_train_imp2, columns=columns))

---


## Realizando a conversão de atributos categóricos para numéricos

Os algoritmos de aprendizado supervisionado implementados em Python, de forma geral, não lidam bem com valores categóricos que estão representados como strings ou caracteres. Existem ainda algoritmos que por padrão, não recebem valores categóricos como entrada (por exemplo, regressão logística e redes neurais). Assim, é preciso codificar os valores categóricos para numéricos. A maioria dos atributos categóricos no conjunto de dados analisado são do tipo nominal: para atributos nominal, é indicado o uso de OneHotEncoder(). No caso de atributos ordinais, é indicado o uso de OrdinalEncoder().


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='if_binary', sparse=False))])

data_pipeline = ColumnTransformer([
    ('numerical', num_pipeline, num_columns),
    ('categorical', cat_pipeline, cat_columns),
    
])

X_train_preproc1 = data_pipeline.fit_transform(X_train)
X_test_preproc1 = data_pipeline.transform(X_test)

In [ ]:
# ajusta nome das columas e mostra dataframe após pré-processamento
columns = np.append(num_columns,data_pipeline.named_transformers_['categorical']['encoder'].get_feature_names_out(cat_columns))
display(pd.DataFrame(X_train_preproc1, columns=columns))

---


## Normalizando os dados

Podemos observar que mesmo após a imputação de valores faltantes e conversão de atributos categóricos para numéricos, o conjunto de dados ainda apresenta uma característica que pode ser crítica para alguns algoritmos de aprendizado: os atributos possuem diferenças de escalas. Assim, é preciso normalizar os valores.

Uma vez que todos os atributos se tornaram numéricos após a execução do pipeline anterior, podemos aplicar a normalização a todos o atributos, independente do ColumnTransformer. 

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))])

cat_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy='most_frequent')),
                         ('encoder', OneHotEncoder(drop='if_binary', sparse=False))])

data_pipeline = ColumnTransformer([
                                   ('numerical', num_pipeline, num_columns),
                                   ('categorical', cat_pipeline, cat_columns)])

norm_pipe = Pipeline([
                 ('data_transform', data_pipeline),
                 ('data_normalize',MinMaxScaler())])


X_train_preproc2 = norm_pipe.fit_transform(X_train)
X_test_preproc2 = norm_pipe.transform(X_test)


In [ ]:
# ajusta nome das columas e mostra dataframe após pré-processamento
columns = np.append(num_columns,norm_pipe[0].named_transformers_['categorical']['encoder'].get_feature_names_out(cat_columns))
display(pd.DataFrame(X_train_preproc2, columns=columns))


É importante notar que para o caso de codificação de valores categóricos com One-Hot Encoder, todos os atributos já estão naturalmente no intervalo [0,1], ou seja, estariam normalizados de acordo com o método MinMaxScaler(). Desta forma, poderíamos ter optado colocar a chamada ao método MinMaxScaler() dentro do pipeline para valores numéricos (num_pipeline). Entretanto, no caso de usarmos o método OrdinalEncoder, ou ainda no caso de querermos aplicar a padronização dos dados (StandardScaler()), a normalização/padronização seria feita sobre todos os atributos conforme fizemos na célula acima.

---


## Agregando ao *Pipeline* o treinamento de modelos

Até o momento, nossos dados passaram pelos seguintes pré-processamentos:


*   Atributos **numéricos**: imputação de valores faltantes e normalização
*   Atributos **categóricos**: imputação de valores faltantes, one-hot encoding, e normalização.

Ao concluir este pipeline, nossos dados de Treino e teste estão prontos para o treinamento de modelos. Abaixo vamos exemplificar como a etapa de treinamento do modelo pode ser incorporando ao pipeline, como o passo final da sequência de preparação e análise dos dados. 

Iremos repetir os códigos da célula anterior (na Seção 'Normalizando os dados') para tornar o código mais claro, embora todos os pipelines/dados salvos em variáveis possam ser reutilizados.*italicized text*


In [ ]:
from sklearn.linear_model import LogisticRegression

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))])

cat_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy='most_frequent')),
                         ('encoder', OneHotEncoder(drop='if_binary', sparse=False))])

data_pipeline = ColumnTransformer([
                                   ('numerical', num_pipeline, num_columns),
                                   ('categorical', cat_pipeline, cat_columns)])

preproc_train_pipe = Pipeline([
                 ('data_transform', data_pipeline),
                 ('data_normalize',MinMaxScaler()),
                 ('model',LogisticRegression())])

## para um pipeline que termina com um modelo a ser treinado, 
## a chamada para "ajuste" aos dados de treinamento ocorre com .fit () e recebe os 
## atributos (X) e a classe (y). Achamada para aplicação do pipeline aos dados de 
##treino/teste ocorre com .predict()

preproc_train_pipe.fit(X_train,y_train)

y_train_pred = preproc_train_pipe.predict(X_train)
y_test_pred = preproc_train_pipe.predict(X_test)


In [ ]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score,accuracy_score,f1_score,ConfusionMatrixDisplay ## para avaliação dos modelos

cm = confusion_matrix(y_test, y_test_pred,labels=preproc_train_pipe[2].classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=preproc_train_pipe[2].classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print('Acurácia: {}'.format(round(accuracy_score(y_test, y_test_pred),3)))
print('Recall: {}'.format(round(recall_score(y_test, y_test_pred,pos_label=1),3)))
print('Precisão: {}'.format(round(precision_score(y_test,y_test_pred,pos_label=1),3)))
print('F1-Score: {}'.format(round(f1_score(y_test,y_test_pred,pos_label=1),3)))

### Uso de pipelines com GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

## Cria o pipeline semelhante ao exemplo anteri
preproc_knn_pipe = Pipeline([
                 ('data_transform', data_pipeline),
                 ('data_normalize',MinMaxScaler()),
                 ('model',KNeighborsClassifier())])

## Cria uma grid de busca para o KNN
## Os hiperparâmetros de um pipeline são especificados como <step name>__<hyperparameter name>
param_grid = {'model__n_neighbors': range(1, 10)}

## Instancia uma Grid Search com o pipeline (incluindo etapas de transformação de dados
## e treinamento do modelo) e 10-fold cross-validation
grid = GridSearchCV(preproc_knn_pipe, param_grid, cv=10,scoring='f1',refit=True)

## Executa a grid Search
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.score(X_test, y_test))

In [ ]:
cm = confusion_matrix(y_test, grid.predict(X_test),labels=preproc_train_pipe[2].classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=preproc_train_pipe[2].classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

### Uso de pipelines com Cross-Validation no Spot-Checking

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.naive_bayes import GaussianNB

preproc_train_pipe = Pipeline([
                 ('data_transform', data_pipeline),
                 ('data_normalize',MinMaxScaler()),
                 ('model',LogisticRegression())])

clfs = []
clfs.append(LogisticRegression())
clfs.append(SVC(kernel='linear'))
clfs.append(SVC(kernel='rbf'))
clfs.append(GaussianNB())
clfs.append(KNeighborsClassifier(n_neighbors=3))
clfs.append(DecisionTreeClassifier())
clfs.append(RandomForestClassifier())
clfs.append(BaggingClassifier())

for classifier in clfs:
    preproc_train_pipe.set_params(model = classifier)
    scores = cross_validate(preproc_train_pipe, X_train, y_train,scoring=['f1','recall','precision'])
    print('---------------------------------')
    print(str(classifier))
    print('-----------------------------------')
    for key, values in scores.items():
            print(key,' mean ', values.mean())
            print(key,' std ', values.std())

## Uso de GridSearch para otimizar escolhas de pré-processamento

Nos exemplos anteriores com uso de Grid Search e k-fold cross-validation, o objetivo era otimizar os hiperparâmetros do algoritmo de aprendizado. Entretanto, também é possível utilizar este processo para otimizar as estratégias de pré-processamento de dados, visando obter melhor poder de generalização.

No exemplo abaixo, iremos demonstrar como utilizar esta estratégia para definir qual o melhor método de normalização dos dados:

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler

# declaramos o pipeline a ser utilizado, dando nomes a cada etapa de forma explícita

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))])

cat_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy='most_frequent')),
                         ('encoder', OneHotEncoder(drop='if_binary', sparse=False))])

data_pipeline = ColumnTransformer([
                                   ('numerical', num_pipeline, num_columns),
                                   ('categorical', cat_pipeline, cat_columns)])

preproc_train_pipe = Pipeline([
                 ('data_transform', data_pipeline),
                 ('scaler',MinMaxScaler()),
                 ('knn',KNeighborsClassifier())])

## A etapa/passo de normalização é denonimado 'scaler'. Podemos atribuir diferentes
## estratégias para este passo em uma grid de hiperparâmetros (incluindo a opção 'passthrough',
## que ignora a etapa). Da mesma forma, iremos definir valores de k-vizinhos mais próximos para avaliar.

param_grid = {'scaler': [MinMaxScaler(), StandardScaler(), RobustScaler(),'passthrough'],
              # we named the second step knn, so we have to use that name here
              'knn__n_neighbors': range(1, 20)}

## Instancia e executa o GridSearch
grid = GridSearchCV(preproc_train_pipe, param_grid, cv=10,scoring='f1',refit=True)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.score(X_test, y_test))

In [ ]:
cm = confusion_matrix(y_test, grid.predict(X_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

Este notebook explorou diferentes estratégias para iniciarmos no pré-processamento de dados (imputação de valores, discretização de dados, codificação de dados e normalização de dados) e demonstrou o uso de Pipelines em Python para criar pipelines de pré-processamento e treinamento facilmente reutilizáveis - incluindo o reuso dentre de loops de GridSearch com validação cruzada.

A partir dos conhecimentos obtidos nessa aula, você já é capaz de implementar algumas etapas iniciais de pré-processamento de dados no ciclo de desenvolvimento de modelos de Aprendizado de Máquina!

## Sua Vez!

Esta atividade prática já será um "aquecimento" para o projeto final da disciplina e visa permitir que os alunos explorem, de forma autônoma, o uso de métodos de pré-processamento e pipelines no desenvolvimento de modelos preditivos em Aprendizado de Máquina.

Os alunos devem:

*   Se organizar em grupos de **até 3 alunos** (interessante que já seja o grupo para realização do projeto final da disciplina).
*   Escolher um conjunto de dados que **não está pronto** para análise por algoritmos de aprendizado supervisionado, apresentando a **necessidade de limpeza e/ou transformação de dados**. Idealmente, selecionar um conjunto de dados que contenha tipos mistos de atributos e que seja de interesse do grupo para realização do projeto final
*   Implementar um pipeline (recomendado o uso de `Pipeline` no scikit-learn) para realizar o pré-processamento de dados utilizando os métodos vistos nesta aula: imputação de valores, codificação de dados categóricos, discretização de dados numéricos, normalização de dados. Eventualmente nem todos serão necessários nos dados, mas sugere-se tentar usar ao menos dois destes. A imputação de valores pode ser utilizada tanto para corrigir valores faltantes como ruídos/outliers.
*   Incluir no pipeline o treinamento de modelos de aprendizado de máquina. Os grupos podem optar por realizar um spot-checking de algoritmos e/ou selecionar um algoritmo específico e realizar a otimização dos seus hiperparâmetros, conforme exemplos deste notebook.
*    Executar o pipeline para os dados selecionados, aplicando as etapas de pré-processamento e treinamento de modelos definidas pelo grupo. 
*    Avaliar o desempenho do modelo final com os dados de teste (evitem data leakage!), reportando matriz de confusão e métrica(s) de desempenho(s) selecionada(s) pelo grupo.

Os grupos devem submeter no Moodle o seu notebook do Google Colab exportado em .pdf e .ypnb, devidamente identificado com os nomes dos integrantes do grupo. Sugere-se que, se possível, os grupos incluam no notebook a ser enviado o link para o Google Colab na nuvem, com saídas salvas, a fim de que a professora e monitores(as) possam acessá-lo em caso de dúvidas na avaliação.
